In [1]:
# Uncomment this if you are using Colab

# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

# import sys
# sys.path.append('/content/drive/My Drive/cs231n-project/')

In [2]:
import numpy as np 
import cvxpy as cp
import time
import matplotlib.pyplot as plt
from lib.utils import relative_error

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, sampler
from torchvision import datasets
import torchvision.transforms as T

USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cpu


In [4]:
BATCH_SIZE = 64
NUM_TRAIN = 50000

In [5]:
# number of datapoints: 60,000
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=T.ToTensor()
)

loader_train = DataLoader(train_data, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

val_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=T.ToTensor()
)

loader_val = DataLoader(val_data, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 60000)))

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=T.ToTensor()
)
print(train_data)
print(test_data)

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [6]:
X_train = train_data.data.numpy()
y_train = train_data.targets.numpy()
X_test = test_data.data.numpy()
y_test = test_data.targets.numpy()

X_val = X_train[-10000:,:,:]
y_val = y_train[-10000:]
X_train = X_train[:-10000,:,:]
y_train = y_train[:-10000]
X_dev = X_train[:30,:,:]
y_dev = y_train[:30]
print('Train')
print(X_train.shape)
print(y_train.shape)
print('Val')
print(X_val.shape)
print(y_val.shape)
print('Test')
print(X_test.shape)
print(y_test.shape)
print('Dev')
print(X_dev.shape)
print(y_dev.shape)

Train
(50000, 28, 28)
(50000,)
Val
(10000, 28, 28)
(10000,)
Test
(10000, 28, 28)
(10000,)
Dev
(30, 28, 28)
(30,)


In [7]:
def flatten(X):
    return X.reshape((X.shape[0], -1))

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

In [8]:
def check_accuracy(loader, model_fn):
    """
    Check the accuracy of a classification model.
    
    Inputs:
    - loader: A DataLoader for the data split we want to check
    - model_fn: A function that performs the forward pass of the model,
      with the signature scores = model_fn(x, params)
    - params: List of PyTorch Tensors giving parameters of the model
    
    Returns: Nothing, but prints the accuracy of the model
    """
    split = 'val' if loader.dataset.train else 'test'
    print('Checking accuracy on the %s set' % split)
    num_correct, num_samples = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.int64)
            # scores = model_fn(x, params)
            scores = model_fn(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f%%)' % (num_correct, num_samples, 100 * acc))

def train(model, optimizer, epochs):
    model = model.to(device=device)
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()
            x = x.to(device=device)  
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if t % 100 == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy(loader_val, model)
                print()


In [9]:
model = nn.Sequential(
    Flatten(),
    nn.Linear(28*28, 32),
    nn.ReLU(),
    nn.Linear(32, 10),
)

optimizer = optim.SGD(
    model.parameters(),
    lr=1e-3,
    momentum=0.9,
    nesterov=True,
)

# train(model, optimizer, epochs=1)

In [10]:
# TODO: Make plot of training accuracy vs time
# TODO: Constrain first layer weights to have unit normal

In [12]:
from lib.convex_fc_vector import Convex_FC_Vector_Solver
solver = Convex_FC_Vector_Solver()
Z, Z_prime, loss = solver.solve(flatten(X_train), y_train, num_classes=10, beta=0.1, max_iters=50000, verbose=True)

Y.shape = (50000, 10)
########
Started Convex FC Vector Solver...
                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 11 12:19:38 PM: Your problem has 12808820 variables, 500040 constraints, and 0 parameters.
(CVXPY) May 11 12:25:27 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 11 12:25:27 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 11 12:25:27 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 11 12:32:45 PM: Compiling problem (target solver=SCS).
(CVXPY) May 11 12:32

In [12]:
Y_hat = solver.predict(flatten(X_train), Z, Z_prime)
y_pred = np.argmax(Y_hat, axis=1)

train_accuracy = np.sum(y_train == y_pred) / y_train.shape[0]
print(f'train_accuracy: {train_accuracy}')

train_accuracy: 1.0
